<a href="https://colab.research.google.com/github/Angelaruiga/appweb-2-/blob/main/Gr%C3%A1fico_Network_Publicaciones(1)(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()
!pip install matplotlib networkx
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import pandas as pd

# Subir archivo desde el sistema local
uploaded = files.upload()

# Asumiendo que sabes el nombre del archivo, por ejemplo 'data_interacciones.csv'
nombre_archivo = next(iter(uploaded))  # Esto obtendrá el nombre del primer archivo subido

# Leer el archivo CSV
df_agrupacion = pd.read_csv(nombre_archivo)

# Mostrar las primeras filas para verificar la carga
print(df_agrupacion)

# Guardar los datos en un archivo CSV para ser utilizado por el script de Streamlit
df_agrupacion.to_csv('data_interacciones.csv', index=False)


Saving data_interacciones.csv to data_interacciones.csv
                                                Grupo1  \
0                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
1                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
2                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
3                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
4                     CO1_LECHUGA SANCHO,ALFONSO MARIA   
..                                                 ...   
155                  IN30_SANCHEZ DEL PINO,MARIA JESUS   
156                  IN30_SANCHEZ DEL PINO,MARIA JESUS   
157  IN35_RODRIGUEZ PIÑERO,MANUEL;CONEJERO GOMEZ,RO...   
158       IN52_RUBIO MARTIN,PATRICIA;LOZANO LOAIZA,EVA   
159       IN52_RUBIO MARTIN,PATRICIA;LOZANO LOAIZA,EVA   

                                                Grupo2  Suma de Interacciones  
0    CO17_AGUILAR DIOSDADO,MANUEL;ARROBA ESPINOSA,A...                      1  
1    CO20_BENAVENTE FERNANDEZ,ISABEL;LUBIAN LOPEZ,S...                      1  
2      

In [ ]:
## Guardar el código de Streamlit en un archivo
%%writefile app.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Cargar los datos
data = pd.read_csv('data_interacciones.csv')

# Función para ajustar las etiquetas de los nodos, poner en mayúsculas y reemplazar información tras "_"
def adjust_labels_column(column):
    adjusted_column = column.apply(lambda x: x.split('_')[0].replace(';', '\n').upper() if ';' in x else x.split('_')[0].upper())
    return adjusted_column

# Ajustar las etiquetas de las columnas 'Grupo1' y 'Grupo2'
data['Grupo1'] = adjust_labels_column(data['Grupo1'])
data['Grupo2'] = adjust_labels_column(data['Grupo2'])

# Calcular el tamaño de los nodos basado en el total de interacciones
node_interactions = data.groupby('Grupo1')['Suma de Interacciones'].sum().to_dict()
node_interactions.update(data.groupby('Grupo2')['Suma de Interacciones'].sum().to_dict())

# Crear un widget de lista desplegable para los grupos
unique_groups = list(set(data['Grupo1']).union(set(data['Grupo2'])))
group_selector = st.selectbox('Selecciona un Grupo:', ['All'] + unique_groups)

# Filtrar los datos según el grupo seleccionado
if group_selector != 'All':
    filtered_data = data[(data['Grupo1'] == group_selector) | (data['Grupo2'] == group_selector)]
else:
    filtered_data = data

# Crear el gráfico de red dirigido
G = nx.DiGraph()
for index, row in filtered_data.iterrows():
    G.add_edge(row['Grupo1'], row['Grupo2'], weight=row['Suma de Interacciones'])

pos = nx.spring_layout(G, k=0.1)  # Ajuste para mejorar la dispersión de los nodos
labels = {node: node for node in G.nodes()}

# Calcular el tamaño de los nodos basado en el total de interacciones
node_size = [node_interactions.get(node, 1) * 500 for node in G.nodes()]  # Aumentado el tamaño de los nodos

# Configurar el Streamlit
st.title("Publicaciones 2023 en colaboración")

# Crear gráfico con matplotlib y mostrarlo en Streamlit
fig, ax = plt.subplots(figsize=(35, 20))  # Aumentado el tamaño del gráfico
nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color='orange', alpha=0.9, ax=ax)
nx.draw_networkx_edges(G, pos, width=0.5, alpha=0.5, edge_color='grey', ax=ax, arrows=True)
nx.draw_networkx_labels(G, pos, labels=labels, font_size=15, font_family="sans-serif", ax=ax)  # Aumentada la fuente de las etiquetas
ax.set_axis_off()
st.pyplot(fig)



Overwriting app.py
